<a href="https://colab.research.google.com/github/crossmodalritual/app/blob/main/io.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import io
import operator  # Importe o módulo operator

# from google.colab import drive, auth  # Remova a autenticação
# from google.auth import default
# import gspread  # Remova gspread

from sklearn.preprocessing import PolynomialFeatures, LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import statsmodels.api as sm

# Novas bibliotecas (e upgrades, se aplicável)
import accelerate  # Para otimização em GPU, se disponível
import google.generativeai as genai  # Para integração com Gemini (opcional)
import diffusers  # Para modelos de difusão (opcional)
import sentence_transformers  # Para embeddings de sentenças (opcional)
import torch  # PyTorch para operações em GPU
import torchvision  # Para tarefas de visão computacional (opcional)
import timm #Efficient Deep Learning PyTorch
import wandb #Weights & Biases

# Constantes e configurações
# DRIVE_MOUNT_PATH = "/content/drive"  # Não é mais necessário montar o Drive
GRAPHPATH = "graficos"  # Salvar localmente (ou modifique para um caminho no Drive, se desejar)
SUMMARYPATH = "sumarios" # Salvar localmente
# CSV_URL = "https://docs.google.com/spreadsheets/d/1GAYrQKB-DF7ea7g0PhxRJN_ItxT-n6eGw7N8QEjUZFg/export?format=csv&gid=1955286066" # Removido
DEFAULT_PALETTE = "Spectral"  # Ou "magma", "viridis", "plasma", "inferno" para paletas neon
FIGSIZE = (10, 6)

# Configuração de estilo para gráficos (fundo preto, cores neon)
sns.set(style="whitegrid")  # Mantém as grades para facilitar a leitura
plt.rcParams.update({
    'figure.facecolor': 'black',
    'axes.facecolor': 'black',
    'axes.edgecolor': 'white',  # Bordas brancas para contraste
    'xtick.color': 'white',
    'ytick.color': 'white',
    'text.color': 'white',
    'legend.facecolor': 'black',
    'axes.labelcolor': 'lime',  # Cor neon para os rótulos dos eixos
    'grid.color': 'gray',     # Cor das grades (mais clara que o fundo)
    'grid.linestyle': '--',   # Estilo da grade
    'figure.titlesize': 20,   # Tamanho do título da figura
    'axes.titlesize': 16,     # Tamanho do título dos eixos
    'axes.labelsize': 12,     # Tamanho dos rótulos dos eixos
})

# Função para criar dados sintéticos (substitui load_data)
def create_synthetic_data(n_samples=300):
    """Cria um DataFrame com dados sintéticos para o estudo."""
    np.random.seed(42)  # Para reprodutibilidade

    # Dados demográficos
    idade = np.random.randint(15, 19, size=n_samples)  # Idade entre 15 e 18 anos
    modelo = np.random.choice(['Tradicional', 'Inovador'], size=n_samples, p=[0.6, 0.4])  # Modelo de ensino
    sexo = np.random.choice(['F', 'M'], size=n_samples, p=[0.5, 0.5])  # Sexo

    # Cria o DataFrame
    df = pd.DataFrame({'Idade': idade, 'Modelo': modelo, 'Sexo': sexo})

    # Simula algumas relações (você pode ajustar isso para refletir melhor sua hipótese)
    # Por exemplo, alunos mais velhos no modelo Inovador podem ter um desempenho ligeiramente melhor
    df['Desempenho'] = 50 + 2 * df['Idade']  # Desempenho base + idade
    df.loc[df['Modelo'] == 'Inovador', 'Desempenho'] += np.random.normal(5, 2, size=sum(df['Modelo'] == 'Inovador')) #Melhor desempenho
    df.loc[df['Sexo'] == 'F', 'Desempenho'] += np.random.normal(2, 1, size=sum(df['Sexo'] == 'F')) #Melhor desempenho
    df['Desempenho'] = np.clip(df['Desempenho'], 0, 100)  # Garante que o desempenho esteja entre 0 e 100

    return df


def preprocess_numeric_cols(df, cols):
    df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')
    return df.dropna(subset=cols)

def save_fig(filename):
    plt.savefig(os.path.join(GRAPHPATH, filename), dpi=300, bbox_inches='tight')
    plt.close()

def create_figure():
    return plt.figure(figsize=FIGSIZE)

# Funções para gerar gráficos de violino
def generate_violinplot(df, x, y, hue, title, filename, split=False, palette=None):
    create_figure()
    if palette is None:
        palette = DEFAULT_PALETTE
    sns.violinplot(x=x, y=y, hue=hue, data=df, split=split, palette=palette)
    plt.title(title)
    save_fig(filename)

# Funções para gerar histogramas
def generate_histplot(df, x, hue, title, filename, bins=20, palette=None):
    create_figure()
    if palette is None:
        palette = DEFAULT_PALETTE
    sns.histplot(data=df, x=x, hue=hue, bins=bins, palette=palette, kde=True) # Adicionado KDE
    plt.title(title)
    save_fig(filename)

# Funções para gerar KDEs
def generate_kdeplot(df, x, hue, title, filename, palette=None):
    create_figure()
    if palette is None:
        palette = DEFAULT_PALETTE
    sns.kdeplot(data=df, x=x, hue=hue, palette=palette, fill=True) # Adicionado preenchimento
    plt.title(title)
    save_fig(filename)

# Funções para gerar gráficos de regressões não lineares
def generate_polynomial_regression(df, x, y, degree, title, filename):
    df = preprocess_numeric_cols(df.copy(), [x, y])
    x_data = df[x].values.reshape(-1, 1)
    y_data = df[y].values.reshape(-1, 1)

    polynomial_features = PolynomialFeatures(degree=degree)
    x_poly = polynomial_features.fit_transform(x_data)

    model = LinearRegression()
    model.fit(x_poly, y_data)

    y_poly_pred = model.predict(x_poly)

    # Ordenar os valores para a linha de regressão ficar correta
    sort_axis = operator.itemgetter(0)
    sorted_zip = sorted(zip(x_data, y_poly_pred), key=sort_axis)
    x_data, y_poly_pred = zip(*sorted_zip)

    plt.scatter(df[x], df[y], color='cyan', label="Dados")  # Pontos em ciano
    plt.plot(x_data, y_poly_pred, color='magenta', label=f"Regressão (Grau {degree})")  # Linha em magenta
    plt.title(title, color='white')
    plt.xlabel(x, color='lime')
    plt.ylabel(y, color='lime')
    plt.legend()
    save_fig(filename)

def generate_logarithmic_regression(df, x, y, title, filename):
    df = preprocess_numeric_cols(df.copy(), [x, y])
    df = df[df[x] > 0]  # Filtra valores de x > 0 para evitar log de 0 ou negativo
    x_data = np.log(df[x]).values.reshape(-1, 1)
    y_data = df[y].values.reshape(-1, 1)

    model = LinearRegression()
    model.fit(x_data, y_data)

    y_log_pred = model.predict(x_data)

     # Ordenar os valores para a linha de regressão ficar correta
    sort_axis = operator.itemgetter(0)
    sorted_zip = sorted(zip(df[x], np.exp(y_log_pred)), key=sort_axis) #x original e y exponencial
    x_data_original, y_log_pred = zip(*sorted_zip)

    plt.scatter(df[x], df[y], color='cyan', label="Dados")
    plt.plot(x_data_original, y_log_pred, color='magenta', label="Regressão Logarítmica")
    plt.title(title, color='white')
    plt.xlabel(x, color='lime')
    plt.ylabel(y, color='lime')
    plt.legend()
    save_fig(filename)

def generate_exponential_regression(df, x, y, title, filename):
    df = preprocess_numeric_cols(df.copy(), [x, y])
    df = df[df[y] > 0]  # Filtra valores de y > 0 para evitar log de 0 ou negativo
    x_data = df[x].values.reshape(-1, 1)
    y_data = np.log(df[y]).values.reshape(-1, 1)

    model = LinearRegression()
    model.fit(x_data, y_data)

    y_exp_pred = np.exp(model.predict(x_data))

    # Ordenar os valores para a linha de regressão ficar correta
    sort_axis = operator.itemgetter(0)
    sorted_zip = sorted(zip(x_data, y_exp_pred), key=sort_axis)
    x_data, y_exp_pred = zip(*sorted_zip)

    plt.scatter(df[x], df[y], color='cyan', label="Dados")
    plt.plot(x_data, y_exp_pred, color='magenta', label="Regressão Exponencial")
    plt.title(title, color='white')
    plt.xlabel(x, color='lime')
    plt.ylabel(y, color='lime')
    plt.legend()
    save_fig(filename)

# Função para gerar sumário estatístico detalhado com intervalo de confiança e valor de p
def generate_statistical_summary_detailed(df, y_var, filename="sumario_detalhado.txt"):
    try:
        X = df[['Idade', 'Sexo_Num', 'Modelo_Num']]
        X = sm.add_constant(X)  # Adiciona a constante para o intercepto
        y = df[y_var]

        model = sm.OLS(y, X).fit()
        summary = model.summary()

        with open(os.path.join(SUMMARYPATH, filename), 'w') as f:
            f.write(summary.as_text())
    except Exception as e:
        print(f"Erro ao gerar o sumário estatístico detalhado: {e}")

# Função principal (main)
if __name__ == "__main__":
    # drive.mount(DRIVE_MOUNT_PATH, force_remount=True) # Removido: Não monta mais o Drive
    os.makedirs(GRAPHPATH, exist_ok=True)  # Cria o diretório localmente
    os.makedirs(SUMMARYPATH, exist_ok=True) # Cria o diretório localmente

    # df = load_data(CSV_URL)  # Removido: agora usa dados sintéticos
    df = create_synthetic_data()


    if df is not None:
        # Transformar colunas categóricas para numéricas
        le_sexo = LabelEncoder()
        df['Sexo_Num'] = le_sexo.fit_transform(df['Sexo'])

        le_modelo = LabelEncoder()
        df['Modelo_Num'] = le_modelo.fit_transform(df['Modelo'])

        # --- Geração dos Gráficos ---

        # Paletas de cores para os gráficos
        paleta1 = ["#00FF00", "#FFA500"]  # Verde e Laranja
        paleta2 = ["#00FF00", "#0000FF", "#FFA500"]  # Verde, Azul e Laranja
        paleta3 = ["#00FF00", "#0000FF", "#FFA500", "#FF00FF"]  # Verde, Azul, Laranja e Rosa

        # Gráficos de violino
        generate_violinplot(df, 'Modelo_Num', 'Idade', 'Sexo_Num', 'Idade por Modelo e Sexo (Bipartido)', 'violin_idade_modelo_sexo_split.png', split=True, palette=paleta1)
        generate_violinplot(df, 'Sexo_Num', 'Idade', 'Modelo_Num', 'Idade por Sexo e Modelo (Bipartido)', 'violin_idade_sexo_modelo_split.png', split=True, palette=paleta2)
        generate_violinplot(df, 'Modelo_Num', 'Idade', 'Sexo_Num', 'Idade por Modelo e Sexo', 'violin_idade_modelo_sexo.png', palette=paleta3)
        generate_violinplot(df, 'Sexo_Num', 'Idade', 'Modelo_Num', 'Idade por Sexo e Modelo', 'violin_idade_sexo_modelo.png', palette=paleta1)

        # Histogramas
        generate_histplot(df, 'Idade', 'Sexo_Num', 'Distribuição de Idade por Sexo', 'hist_idade_sexo.png', bins=20, palette=paleta2)
        generate_histplot(df, 'Idade', None, 'Distribuição de Idade', 'hist_idade_geral.png', bins=20, palette=paleta3)

        # KDEs
        generate_kdeplot(df, 'Idade', 'Modelo_Num', 'Densidade de Idade por Modelo', 'kde_idade_modelo.png', palette=paleta1)
        generate_kdeplot(df, 'Idade', 'Sexo_Num', 'Densidade de Idade por Sexo', 'kde_idade_sexo.png', palette=paleta2)

        # Gráficos de regressões não lineares (com tratamento de erros e avisos)
        try:
            generate_polynomial_regression(df, 'Idade', 'Sexo_Num', 2, 'Regressão Polinomial (Grau 2) - Idade vs. Sexo', 'poly_idade_sexo_2.png')
            generate_polynomial_regression(df, 'Idade', 'Modelo_Num', 3, 'Regressão Polinomial (Grau 3) - Idade vs. Modelo', 'poly_idade_modelo_3.png')
            generate_logarithmic_regression(df, 'Idade', 'Sexo_Num', 'Regressão Logarítmica - Idade vs. Sexo', 'log_idade_sexo.png')
            generate_exponential_regression(df, 'Idade', 'Modelo_Num', 'Regressão Exponencial - Idade vs. Modelo', 'exp_idade_modelo.png')

        except ValueError as ve:
            print(f"Erro ao gerar regressão: {ve}. Verifique se os dados são adequados para o tipo de regressão.")
        except Exception as e:
            print(f"Erro inesperado ao gerar regressão: {e}")

        # Sumário estatístico detalhado
        generate_statistical_summary_detailed(df, 'Desempenho')

        print("Gráficos e sumário gerados com sucesso!")

<ipython-input-6-3a2172b5fff1>:70: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[86.04282751 91.28750809 85.67699281 84.16942417 88.26556373 93.54138572
 89.36373251 85.49644117 86.0812782  89.30031126 88.66067163 89.28783235
 91.14313247 86.04468511 85.95795965 91.66732421 91.07507989 85.9799672
 90.46025013 89.04247257 88.11141348 87.75460099 88.51397723 87.15566935
 92.73921184 91.71127572 85.82686981 92.75359163 83.4524216  86.51069059
 81.4425595  93.99208862 88.30873131 86.88883066 91.55993725 86.74902191
 95.89150396 89.25844236 89.21878959 92.45153325 91.96201846 91.44776805
 85.41905109 87.94293671 94.76404899 89.69084009 88.18637325 85.97756865
 85.02079036 90.74842616 91.11144982 91.18838304 85.61507074 94.05910064
 84.6839842  86.14623786 88.97579125 81.69028666 88.64634117 89.14663593
 84.4200782  82.40984246 90.3284306  94.33804305 90.4808173  87.99371409
 88.50851387 84.45455286 83.606226

Gráficos e sumário gerados com sucesso!
